# Meet the meat

## Abstract

With increasingly dire climate change forecasts, concerned individuals are asking how they can minimize their carbon footprint. Recent research suggests that reducing one's consumption of meat, in particular beef, is one of the highest impact actions an individual can take. To examine this topic, we will explore the popularity and prevalence of meat in recipes. Specifically, we plan to extract the ingredients from a recipe database and calculate the carbon footprint of recipes

Finally, we hope to directly relate this data to the issue of climate change by estimating a rating reflecting the carbon footprint of meat in recipes and the environmental impact of consumers' diets.

### Imports and libraries

In [1]:
# Import libraries
import re
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import os, os.path as osp

In [2]:
DATA_FOLDER='data'
SAMPLE_DATA_FOLDER = DATA_FOLDER + '/sample_400/'

## Data extraction and cleaning

Our recipe dataset contains recipes from the [From Cookies to Cooks](http://infolab.stanford.edu/~west1/from-cookies-to-cooks/), combining recipes from 14 high-traffic websites. We start by extracting all the information we want from the HTML files, that is: title, ingredients and meat or animal protein ingredients, tags, ratings in order to explore the recipes in more detail.


#### Recipe webpage scraping
The websites' HTML sources are rich in information. However, the information we wantfrom these pages is rather limited. We extract the information we need from the websites, clean and pre-process the data and save it as a CSV file for easy retrieval in further processing.

In [30]:
def analyse_page(soup, page):
    """
    Input: 
        soup
        page: 'allrecipes', 'epicurious', 'food_network', 'food_com', 'betty_crocker', 'my_recipes' , others not implemented yet
    
    Output:
        tags = list of tags assigned to the recipe
        ings = list of ingredients
    """
    ings = []
    tags = []
    if page == 'allrecipes':
        # Extract tags
        tag_wrappers = soup.find_all(itemprop="recipeCategory")
        for tag in tag_wrappers:
            tags.append(tag['content'])           
        # Extract ingredients
        ing_wrap=soup.find_all('li', class_="plaincharacterwrap ingredient")
        if ing_wrap:
            for ing in ing_wrap:
                ings.append(ing.getText())
        else:##?
            ing_wrap=soup.find_all(itemprop="recipeIngredient")

            for ing in ing_wrap:
                ings.append(ing.getText())
            if not ing_wrap:
                print('alternative format needed for Allrecipes')
        
        
    elif page == 'epicurious':       
        # Extract tags
        tag_wrappers = soup.find_all(itemprop="recipeCuisine")
        for tag in tag_wrappers:
            tags.append(tag.getText())    
        tag_wrappers = soup.find_all(itemprop="recipeCategory")
        for tag in tag_wrappers:
            tags.append(tag.getText())        
        # Extract ingredients
        ing_wrap=soup.find('div', id="ingredients")
        for ing in ing_wrap:
            ings.append(ing.string)
      
    
    elif page == 'food_network':  
        # Extract tags
        tag_wrappers = soup.find_all(class_="btn grey-tags")        
        for tag in tag_wrappers:
            tags.append(tag.getText())      
        # Extract ingredients
        ing_wrap=soup.find_all('li',class_='ingredient')
        for ing in ing_wrap:
            ings.append(ing.text)

    elif page == 'food_com':      
        # Extract tags
            #not found          
        # Extract ingredients
        ing_wrap=soup.find_all('li', class_="ingredient")
        if ing_wrap:
            for ing in ing_wrap:
                ings.append((ing.find('span',class_='value').text+ ' '+ing.find('span',class_='type').text + ' ' + ing.find('span', class_='name').text))
        else:
            ing_wrap=soup.find_all(class_="name")
            for ing in ing_wrap:
                ings.append(ing.getText())
    
    elif page == 'betty_crocker':   
        # Extract tags
            #not found    
        # Extract ingredients
        ing_wrap=soup.find_all('dl', class_='ingredient')
        for ing in ing_wrap:
            ings.append(ing.getText())
    
    
    elif page == 'my_recipes':
        # Extract tags
        tag_wrappers = soup.find_all(itemprop="recipeType")
        for tag in tag_wrappers:
            tags.append(tag.getText())  
        # Extract ingredients
        ing_wrap=soup.find_all(itemprop="ingredient")
        for ing in ing_wrap:
            ings.append(ing.text)
        
    #other websites    
        # Extract tags   
        # Extract ingredients 
        
    if not ing_wrap:  #return warning if website is recognized but format/data extraction is not successful
        print('*******')
        print('NEED NEW FORMAT')  
        print('*******')
    
    #if not tags:
        #print('no tags found :( ')
        
    return tags, ings#, ing_amnts



In [31]:
def find_website(soup):
    """
    Finds if the page is a recipe and which website it comes from
    """
    is_recipe = True
    
    if 'Allrecipes' in soup.title.string:
        website = 'allrecipes'               
              
    elif 'Epicurious' in soup.title.string:
        website = 'epicurious'
    
    elif 'Food Network' in soup.title.string:
        website = 'food_network'
        
    elif 'Food.com' in soup.title.string:
        website == 'food_com'
    
    elif 'Betty Crocker' in soup.title.string:
        website = 'betty_crocker'
               
    elif 'MyRecipes' in soup.title.string:
        website = 'my_recipes'

    else:
        website = 'not found'
        is_recipe = False
        
    return is_recipe, website

#### Quantity extraction and conversion
The amounts of each ingredients are expressed in many different units (imperial or metric) depending on the websites, and even on the recipes. Once we have extracted the ingredients and amounts, we need to convert all different quantities to one single weight unit (fixed to kilograms) in order to process the carbon footprint of selected ingredients.

In [32]:
def check_quantity(quant_str):
    """
    Cleans input string and extracts numerical values
    Outputs cleaned string, array of numerical values and sum of numerical values
    """
    quant_str=quant_str.replace("½",".5")
    quant_str=quant_str.replace("1/2",".5")
    quant_str=quant_str.replace("1/3", '.33')
    quant_str=quant_str.replace('1/4','.25')
    quant_str=quant_str.replace('3/4','.75')
    quant_vals=re.findall(r"[-+]?\d*\.\d+|\d+", quant_str)
    total_quant=np.sum([float(i) for i in quant_vals])
    
    return quant_str, quant_vals, total_quant


def convert_to_kg(quant, unit):
    """
    Converts any input unit (kg, lb, grams, ounces) to kilograms
    """
    
    if (unit=='kilogram') or (unit=='kg'):
        amnt_kg=quant
        #print(quant,'kg')
    elif (unit=='pound') or (unit=='lb') or (unit=='lbs') or (unit=='pounds'):
        amnt_kg=quant/2.205
        #print(amnt_kg,'kg')
    elif(unit=='g') or (unit=='gram') or (unit =='grams'):
        amnt_kg=quant/1000
        #print(amnt_kg,'kg')     
    elif(unit=='oz') or (unit=='ounce'):
        amnt_kg=quant/35.274
        #print(amnt_kg, 'kg')
        
    return(amnt_kg)

def contains_meat_ingredients(ings_in, meat_products_in):
    contains_meat=False
    meat_ingredients=[]
    meat_ingredients_str=[]
    #meat_ingredients=[False]*len(meat_products_in)
    for i in ings_in:
        for meat_product in meat_products_in:
            if i != None:
                if meat_product in i.casefold(): 
                    contains_meat=True
                    meat_ingredients.append(meat_product) 
                    meat_ingredients_str.append(i)
                    
    return contains_meat, meat_ingredients, meat_ingredients_str

def extract_meat(meat_ings_str):
    """
    Inputs: 
    ings_in= list of ingredients (and quantities)
     
    Outputs:
    ing_amnt_out = list of corresponding quanities of meat ingredients in kg (=0 if unit not recognized)
    
    """

    ing_amnt_out=[]
   
    #extract amount from string and convert to kg
    for meat_i in meat_ings_str:
        meat_i_quant_kg=0
        meat_i, quantity_vals, total_quantity=check_quantity(meat_i) #pass string, return cleaned string and total quantity

        for u in units: 
            if u in meat_i:
                meat_i_quant_kg = convert_to_kg(total_quantity,u)
        ing_amnt_out.append(meat_i_quant_kg)
        #if meat_i_quant_kg==0:
        #    print('Units not recognized for: '+meat_i)
                    
    return ing_amnt_out

#### Define carbon footprint of meat ingredients
Animal agriculture is one of the leading sources of the carbon-impact of a recipe. We start by assigning a carbon footprint to each meat ingredient and could later on extend it to other animal products. 
The functions below assign a carbon footprint to each meat ingredient of the recipes.

Source of data: [GreenEatz](https://www.greeneatz.com/foods-carbon-footprint.html)

In [33]:
#Load data from xls file
carbon_footprint = pd.read_excel('data/carbon_footprint_protein.xls', sheet_name='meat_dairy_eggs', index_col=0)
carbon_footprint

,Food,CO2 Kilos Equivalent,Car Miles Equivalent
Rank,,,
1,Lamb,39.2,91
2,Beef,27.0,63
3,Cheese,13.5,31
4,Pork,12.1,28
5,Turkey,10.9,25
6,Chicken,6.9,16
7,Tuna,6.1,14
8,Eggs,4.8,11


In [36]:
#List of meat ingredients
meat_products = carbon_footprint['Food'].tolist()
#same list copied without caps
meat_products = ['lamb', 'beef', 'cheese', 'pork', 'turkey', 'chicken', 'tuna', 'egg']

In [9]:
units = ['pounds','grams','oz','ounces','kg','kilograms','lbs' ]

In [10]:
# calculate carbon footprint
#input ingredients and amounts
#output carbon footprint
def carbon_fp (l):
    """
    takes a list of ingredients contributing to co2 and returns carbon footprint
    """
    c=len(l)
    return c

### Data extraction and cleaning loop
Below we extract the data from the recipes of our html dataset and save it in dataframes. Our goal here is to extract the ingredients and assign a carbon-impact rating to the highest impact ingredients (meat or animal protein) in the recipes.

To extract protein-rich ingredients from animal source in order to calculate the main carbon footprint of the recipe, we use an extra database listing the main protein sources and carbon impact. Source of data: [GreenEatz](https://www.greeneatz.com/foods-carbon-footprint.html)

In [37]:
#Loop for all recipes in folder
# data has following row structure
# RecipeName as Identifier - bool contains_meat - list of co2 ingredients - carbonFootprint - ingredients
data=[]
step=0

verbose = 0 #verbose outputs

for filename in os.listdir(SAMPLE_DATA_FOLDER):
    with open(SAMPLE_DATA_FOLDER+filename) as f:
        isTrue=False
        count_exceptions=0
        print('----------------------------------')
        try:
            page = f.read()
            soup = BeautifulSoup(page, 'html.parser')
            
            #check webpage and extract ingredients if recognised as recipe
            is_recipe, website = find_website(soup)
            print('This recipe is',soup.title,' from: '+website)            
            
            if is_recipe:

                #tags, ingredients = analyse_page(soup, website)
                tags, ingredients= analyse_page(soup, website)
                
                if ingredients:
                    
                    
                    has_meat, meat_ingredients, meat_ingredients_str = contains_meat_ingredients(ingredients, meat_products)
                    
                    if has_meat:

                        #Extract all ingredients (and ingredient amounts if available)
                        #Extract meat ingredients and quantities in kg
                        #print('ingredient ammount = ', ingredient_amounts)
                        ingredient_quant_kg = extract_meat(meat_ingredients_str)
                        print('does this recipe contain meat? ', contains_meat)
                        print('ingredients = ',ingredients)
                        print('meat ingredients=', meat_ingredients_str)
                        print('ingredient_quantity (kg)= ',ingredient_quant)
                        

                    if verbose: 
                        print('Recipe Analysed: '+soup.title.string)
                        
                        print('contains meat:'+str(has_meat))
                        print(meat_ingredients)
                        
                        print('{0} Ingredients: '.format(len(ingredients)))
                        print(ingredients)

                        print('{0} tags:'.format(len(tags)))
                        print(tags)
                            
                data.append([soup.title.string, has_meat, meat_ingredients, tags])

            #else:
                #print('not a recipe')
        except:
            count_exceptions=count_exceptions+1
            #print('Exception')
    step=step+1
    
    if verbose: 
        print('-------------------------------------')
        
    if step>=100:
        break

column_labels=['Recipe Title', 'Has meat', 'Meat types', 'Tags']#missing: 'Carbon footprint', 'Rating', 'Tags'
recipes_df = pd.DataFrame(data, columns = column_labels)

#save the data as csv for in depth analysis
#recipes_df.to_csv(DATA_FOLDER+'/recipes_data')

recipes_df

----------------------------------
This recipe is <title>
	Chicken Breast Cutlets with Artichokes and Capers Recipe - Allrecipes.com
</title>  from: allrecipes
----------------------------------
This recipe is <title>
	Best Ever Popcorn Balls Recipe - Allrecipes.com
</title>  from: allrecipes
----------------------------------
This recipe is <title>Pumpkin Oatmeal Recipe : Aarti Sequeira : Recipes : Food Network</title>  from: food_network
----------------------------------
This recipe is <title>
	Green Bean Casserole  Recipe from Betty Crocker
</title>  from: betty_crocker
----------------------------------
This recipe is <title>Lemon Wasabi Sauce Recipes | Yummly</title>  from: not found
----------------------------------
This recipe is <title>
	Orange Cream Cheese Frosting Recipe - Allrecipes.com
</title>  from: allrecipes
----------------------------------
This recipe is <title>Orange Curd Recipe : Ina Garten : Recipes : Food Network</title>  from: food_network
--------------------

----------------------------------
This recipe is <title>Appetizer Recipes By Kraft Foods - Cold &amp; Hot Appetizers: Easy Snack Recipes &amp; Party Appetizers</title>  from: not found
----------------------------------
This recipe is <title>Crockpot Oatmeal Recipe - Slow Cooker Steel Cut Oats Breakfast Recipe</title>  from: not found
----------------------------------
This recipe is <title>Scallop Fakes and Scallop Types</title>  from: not found
----------------------------------
----------------------------------
This recipe is <title>Southern Pecan Pie Recipe</title>  from: not found
----------------------------------
This recipe is <title>
	Liver Pate Recipe - Allrecipes.com
</title>  from: allrecipes
----------------------------------
----------------------------------
This recipe is <title>
	Garlic Mashed Potatoes Recipe - Allrecipes.com
</title>  from: allrecipes
----------------------------------
This recipe is <title>









Black Bean and Sweet Potato Stew with Chilies and

,Recipe Title,Has meat,Meat types,Tags
0,Best Ever Popcorn Balls Recipe - Allrecipes....,False,[],[]
1,Pumpkin Oatmeal Recipe : Aarti Sequeira : Reci...,False,[],[]
2,Green Bean Casserole Recipe from Betty Croc...,False,[],[]
3,Perfect Fudge Article - Allrecipes.com,True,[egg],[]
4,Pumpkin Oatmeal Recipe - Allrecipes.com,False,[],[]
5,Baked Asparagus with Balsamic Butter Sauce R...,False,[],[]
6,Herb and Spice Guide : Recipes and Cooking : F...,True,"[chicken, chicken]",[]
7,Fruit Dip Recipes - Allrecipes.com,True,"[chicken, chicken]",[]
8,Fry Bread I Recipe - Allrecipes.com,False,[],[]
9,Espresso and Mascarpone Icebox Cake ...,False,[],[]
